#**Lab Assignment 4: Data Preparation with Pandas**##

##**Instructions**

Use the commands presented in the previous sections to perform the following DataFrame manipulations using the a datset published by the **New York City Department of Parks and Recreation (CC0 1.0 Universal Public Domain Dedication).**


---


You will be graded based on whether your Colab notebook can produce .csv file from the DataFrame at the end of this assignment.As such, please submit both your Google Colab notebook and the .csv file it produced.

#**Daniela Cebotari LAB 4, PANDAS**#

### **Solution**

First off, I import the pandas library.

In [ ]:
import pandas as pd

Then, the next step is importing the given dataset:

In [ ]:
lab1_df=pd.read_csv('https://data.cityofnewyork.us/api/views/5rq2-4hqu/rows.csv?accessType=DOWNLOAD')

We can start by using head() to view just the beginning of the file.

In [ ]:
lab1_df.head()

,created_at,tree_id,block_id,the_geom,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,...,st_assem,st_senate,nta,nta_name,boro_ct,state,Latitude,longitude,x_sp,y_sp
0,08/27/2015,180683,348711,POINT (-73.84421521958048 40.723091773924274),3,0,OnCurb,Alive,Fair,Acer rubrum,...,28,16,QN17,Forest Hills,4073900,New York,40.723092,-73.844215,1.027431e+06,202756.768749
1,09/03/2015,200540,315986,POINT (-73.81867945834878 40.79411066708779),21,0,OnCurb,Alive,Fair,Quercus palustris,...,27,11,QN49,Whitestone,4097300,New York,40.794111,-73.818679,1.034456e+06,228644.837379
2,09/05/2015,204026,218365,POINT (-73.93660770459083 40.717580740099116),3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,...,50,18,BK90,East Williamsburg,3044900,New York,40.717581,-73.936608,1.001823e+06,200716.891267
3,09/05/2015,204337,217969,POINT (-73.93445615919741 40.713537494833226),10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,...,53,18,BK90,East Williamsburg,3044900,New York,40.713537,-73.934456,1.002420e+06,199244.253136
4,08/30/2015,189565,223043,POINT (-73.97597938483258 40.66677775537875),21,0,OnCurb,Alive,Good,Tilia americana,...,44,21,BK37,Park Slope-Gowanus,3016500,New York,40.666778,-73.975979,9.909138e+05,182202.425999


###**To start, create a Python list containing the name of the columns of interest.**###

Those being: ('*spc_latin*'), common name ('*spc_common*'), '*status*' (values being 'Alive', 'Dead', or 'Stump'), '*health*', and diameter ('*tree_dbh*' for intact trees & '*stump_diam*' for stumps).



In [ ]:
columnsOfInterest=['spc_latin', 'spc_common', 'status', 'health', 'tree_dbh', 'stump_diam']

###**Then, create a new DataFrame called '*lab1_df_reduced*' that only contains those columns.**###

In [ ]:
lab1_df_reduced = lab1_df[columnsOfInterest]
lab1_df_reduced

,spc_latin,spc_common,status,health,tree_dbh,stump_diam
0,Acer rubrum,red maple,Alive,Fair,3,0
1,Quercus palustris,pin oak,Alive,Fair,21,0
2,Gleditsia triacanthos var. inermis,honeylocust,Alive,Good,3,0
3,Gleditsia triacanthos var. inermis,honeylocust,Alive,Good,10,0
4,Tilia americana,American linden,Alive,Good,21,0
...,...,...,...,...,...,...
683783,Quercus palustris,pin oak,Alive,Good,25,0
683784,Cladrastis kentukea,Kentucky yellowwood,Alive,Good,7,0
683785,Acer rubrum,red maple,Alive,Good,12,0
683786,Acer rubrum,red maple,Alive,Good,9,0


###**Next, we will want to scan our new DataFrame for NaN values that may hinder later analyses.**###

In [ ]:
nullValue_counts=lab1_df_reduced.isnull().sum()
nullValue_counts

spc_latin     31619
spc_common    31619
status            0
health        31616
tree_dbh          0
stump_diam        0
dtype: int64

Based on this, we can see that there are 31,619 NaN values in the DataFrame's '*health*' and name-related columns. Let's investigate futher by trying to figure out why that is.

We'll do so by selecting the rows that have NaNs from our reduced DataFrame with the next command.

( Note that below, '|' means "or" )

In [ ]:
NaNsOnly=lab1_df_reduced.loc[(lab1_df_reduced['spc_latin'].isnull() ) | (lab1_df_reduced['spc_common'].isnull() ) | (lab1_df_reduced['health'].isnull() ) ]
NaNsOnly.head()

,spc_latin,spc_common,status,health,tree_dbh,stump_diam
61,NaN,NaN,Dead,NaN,2,0
307,NaN,NaN,Stump,NaN,0,9
370,NaN,NaN,Dead,NaN,4,0
494,NaN,NaN,Stump,NaN,0,17
556,NaN,NaN,Dead,NaN,2,0


What stands out the most from this new DataFrame is that the status of the first five rows have changed from '*Alive*' to *'Dead'* and *'Stump'*. Based on this, it is starting to seem like there were issues identifying the species and health of dead trees and stumps.

It also seems unlikely that the '*status*' column changed by chance since otherwise that'd suggest New York City could be largely filled with dead trees and stumps.

Let's run one more command so we can gather more evidence to access whether identification issues were the source of our NaNs.

In [ ]:
pd.value_counts(NaNsOnly['status'])

status
Stump    17654
Dead     13961
Alive        6
Name: count, dtype: int64

Based on this evidence, it is now safe to assume that our reduced DataFrame's NaNs are mostly not random mistakes but instead due to trouble identifying physically obscured trees.

Assuming this, we can now decide how we want to deal with the NaN values as we explore different questions about the census.

Let's obtain the mean diameter for every tree, including those which were dead and/or unidentified.  

When calculating the mean here, we should be warry of columns for which no diameter information was recorded as including them in our calculations will render them incorrect.

Running the following command, we can see that 278 of such rows exist.

In [ ]:
lab1_df_reduced.loc[(lab1_df_reduced['tree_dbh'] == 0) & (lab1_df_reduced['stump_diam'] == 0)]

,spc_latin,spc_common,status,health,tree_dbh,stump_diam
644,NaN,NaN,Dead,NaN,0,0
3647,NaN,NaN,Dead,NaN,0,0
5291,Cotinus coggygria,smoketree,Alive,Good,0,0
6029,NaN,NaN,Dead,NaN,0,0
6114,NaN,NaN,Dead,NaN,0,0
...,...,...,...,...,...,...
677527,NaN,NaN,Dead,NaN,0,0
677676,NaN,NaN,Dead,NaN,0,0
677698,NaN,NaN,Dead,NaN,0,0
680500,Acer palmatum,Japanese maple,Alive,Good,0,0


###**Filter out these rows and save the new DataFrame under the name 'lab1_df_reduced_zerosFiltered'.**

By using the command below, you can confirm the this filtered DataFrame is exactly 278 rows shorter than before.

In [ ]:
lab1_df_reduced_zeroesFiltered =  lab1_df_reduced.loc[(lab1_df_reduced['tree_dbh'] != 0) | (lab1_df_reduced['stump_diam'] != 0)]
lab1_df_reduced_zeroesFiltered

,spc_latin,spc_common,status,health,tree_dbh,stump_diam
0,Acer rubrum,red maple,Alive,Fair,3,0
1,Quercus palustris,pin oak,Alive,Fair,21,0
2,Gleditsia triacanthos var. inermis,honeylocust,Alive,Good,3,0
3,Gleditsia triacanthos var. inermis,honeylocust,Alive,Good,10,0
4,Tilia americana,American linden,Alive,Good,21,0
...,...,...,...,...,...,...
683783,Quercus palustris,pin oak,Alive,Good,25,0
683784,Cladrastis kentukea,Kentucky yellowwood,Alive,Good,7,0
683785,Acer rubrum,red maple,Alive,Good,12,0
683786,Acer rubrum,red maple,Alive,Good,9,0


In [ ]:
len(lab1_df_reduced_zeroesFiltered)

683510

Next, peform the following steps:


1.   Group the rows of the filtered DataFrame by the columns '_status_', '_spc_latin_', '_spc_common_', and '_health_'.
2.   Add 'dropna=False' as an argument to groupby().
3.   Use mean() to get the mean tree diameter for each group.
4.   Save this as a DataFrame named 'lab1_df_reduced_zerosFiltered_grouped



In [ ]:
groupedByColumns = ['status', 'spc_latin', 'spc_common', 'health']

In [ ]:
lab1_df_reduced_zerosFiltered_grouped = lab1_df_reduced_zeroesFiltered.groupby(groupedByColumns, dropna = False).mean()
lab1_df_reduced_zerosFiltered_grouped

tree_dbh  \
status spc_latin                          spc_common    health              
Alive  Acer                               maple         Fair    11.903991   
                                                        Good    11.778865   
                                                        Poor    10.509434   
       Acer buergerianum                  trident maple Fair     9.750000   
                                                        Good     7.724138   
...                                                                   ...   
       NaN                                NaN           Good    13.000000   
                                                        Poor    11.000000   
Dead   Gleditsia triacanthos var. inermis honeylocust   NaN     12.000000   
       NaN                                NaN           NaN      5.492017   
Stump  NaN                                NaN           NaN      0.000000   

                                                                stump_diam  
status spc_latin                          spc_common    health              
Alive  Acer                               maple         Fair      0.000000  
                                                        Good      0.000000  
                                                        Poor      0.000000  
       Acer buergerianum                  trident maple Fair      0.000000  
                                                        Good      0.000000  
...                                                                    ...  
       NaN                                NaN           Good      0.000000  
                                                        Poor      0.000000  
Dead   Gleditsia triacanthos var. inermis honeylocust   NaN       0.000000  
       NaN                                NaN           NaN       0.000000  
Stump  NaN                                NaN           NaN      16.750481  

[398 rows x 2 columns]

While we've come quite far from our original DataFrame, we still want to modify this one.

###**To get the DataFrame ready for that, reset its index columns.**###

In [ ]:
lab1_df_reduced_zerosFiltered_grouped = lab1_df_reduced_zerosFiltered_grouped.reset_index()
lab1_df_reduced_zerosFiltered_grouped

,status,spc_latin,spc_common,health,tree_dbh,stump_diam
0,Alive,Acer,maple,Fair,11.903991,0.000000
1,Alive,Acer,maple,Good,11.778865,0.000000
2,Alive,Acer,maple,Poor,10.509434,0.000000
3,Alive,Acer buergerianum,trident maple,Fair,9.750000,0.000000
4,Alive,Acer buergerianum,trident maple,Good,7.724138,0.000000
...,...,...,...,...,...,...
393,Alive,NaN,NaN,Good,13.000000,0.000000
394,Alive,NaN,NaN,Poor,11.000000,0.000000
395,Dead,Gleditsia triacanthos var. inermis,honeylocust,NaN,12.000000,0.000000
396,Dead,NaN,NaN,NaN,5.492017,0.000000


###**Create a new column called '*diameter*' that combines the 'tree_dbh' and 'stump_diam' columns.**###

In [ ]:
lab1_df_reduced_zerosFiltered_grouped["diameter"] = lab1_df_reduced_zerosFiltered_grouped["tree_dbh"] + lab1_df_reduced_zerosFiltered_grouped["stump_diam"]
lab1_df_reduced_zerosFiltered_grouped

,status,spc_latin,spc_common,health,tree_dbh,stump_diam,diameter
0,Alive,Acer,maple,Fair,11.903991,0.000000,11.903991
1,Alive,Acer,maple,Good,11.778865,0.000000,11.778865
2,Alive,Acer,maple,Poor,10.509434,0.000000,10.509434
3,Alive,Acer buergerianum,trident maple,Fair,9.750000,0.000000,9.750000
4,Alive,Acer buergerianum,trident maple,Good,7.724138,0.000000,7.724138
...,...,...,...,...,...,...,...
393,Alive,NaN,NaN,Good,13.000000,0.000000,13.000000
394,Alive,NaN,NaN,Poor,11.000000,0.000000,11.000000
395,Dead,Gleditsia triacanthos var. inermis,honeylocust,NaN,12.000000,0.000000,12.000000
396,Dead,NaN,NaN,NaN,5.492017,0.000000,5.492017


Since all rows have a non-zero value in one column but not both, it will be safe to sum the two columns.

###**Run the following command to remove the '*tree_dbh*' and '*stump_diam*' columns:**###

In [ ]:
lab1_df_reduced_zerosFiltered_grouped.drop(columns=['tree_dbh', 'stump_diam'], inplace=True)
lab1_df_reduced_zerosFiltered_grouped

,status,spc_latin,spc_common,health,diameter
0,Alive,Acer,maple,Fair,11.903991
1,Alive,Acer,maple,Good,11.778865
2,Alive,Acer,maple,Poor,10.509434
3,Alive,Acer buergerianum,trident maple,Fair,9.750000
4,Alive,Acer buergerianum,trident maple,Good,7.724138
...,...,...,...,...,...
393,Alive,NaN,NaN,Good,13.000000
394,Alive,NaN,NaN,Poor,11.000000
395,Dead,Gleditsia triacanthos var. inermis,honeylocust,NaN,12.000000
396,Dead,NaN,NaN,NaN,5.492017


Our final modification to this DataFrame will be to add counts for each row. To start:

###**Assign each row in the reduced but ungrouped DataFrame ('*lab1_df_reduced*') with a count of 1 using the code below:**###

In [ ]:
lab1_df_reduced_counts=lab1_df_reduced.assign(count = 1)
lab1_df_reduced_counts.head()

,spc_latin,spc_common,status,health,tree_dbh,stump_diam,count
0,Acer rubrum,red maple,Alive,Fair,3,0,1
1,Quercus palustris,pin oak,Alive,Fair,21,0,1
2,Gleditsia triacanthos var. inermis,honeylocust,Alive,Good,3,0,1
3,Gleditsia triacanthos var. inermis,honeylocust,Alive,Good,10,0,1
4,Tilia americana,American linden,Alive,Good,21,0,1


Next, perform the following steps on the *'lab1_df_reduced_counts'* DataFrame.



1.   Group the rows of the filtered DataFrame by the columns '_status_', '_spc_latin_', '_spc_common_', and '_health_'.
2.   Add 'dropna=False' as an argument to groupby().
3. Use sum() to add up the counts for each group.



In [ ]:
lab1_df_reduced_counts_grouped = lab1_df_reduced_counts.groupby(["status", "spc_latin", "spc_common", "health"], dropna=False).sum()
lab1_df_reduced_counts_grouped

tree_dbh  \
status spc_latin                          spc_common    health             
Alive  Acer                               maple         Fair       20582   
                                                        Good       55396   
                                                        Poor        6684   
       Acer buergerianum                  trident maple Fair         195   
                                                        Good         672   
...                                                                  ...   
       NaN                                NaN           Good          52   
                                                        Poor          11   
Dead   Gleditsia triacanthos var. inermis honeylocust   NaN           12   
       NaN                                NaN           NaN        76361   
Stump  NaN                                NaN           NaN            0   

                                                                stump_diam  \
status spc_latin                          spc_common    health               
Alive  Acer                               maple         Fair             0   
                                                        Good             0   
                                                        Poor             0   
       Acer buergerianum                  trident maple Fair             0   
                                                        Good             0   
...                                                                    ...   
       NaN                                NaN           Good             0   
                                                        Poor             0   
Dead   Gleditsia triacanthos var. inermis honeylocust   NaN              0   
       NaN                                NaN           NaN              0   
Stump  NaN                                NaN           NaN         295713   

                                                                count  
status spc_latin                          spc_common    health         
Alive  Acer                               maple         Fair     1733  
                                                        Good     4708  
                                                        Poor      639  
       Acer buergerianum                  trident maple Fair       20  
                                                        Good       87  
...                                                               ...  
       NaN                                NaN           Good        4  
                                                        Poor        1  
Dead   Gleditsia triacanthos var. inermis honeylocust   NaN         1  
       NaN                                NaN           NaN     13960  
Stump  NaN                                NaN           NaN     17654  

[398 rows x 3 columns]

###**4. Save this as a DataFrame named 'lab1_df_reduced_counts_grouped'**###


With the '*count*' column set up, the heavy-lifting has been done. To clean up,

In [ ]:
lab1_df_reduced_counts_grouped.drop(columns=['tree_dbh','stump_diam'], inplace=True)
lab1_df_reduced_counts_grouped=lab1_df_reduced_counts_grouped.reset_index()
lab1_df_reduced_counts_grouped.head()

,status,spc_latin,spc_common,health,count
0,Alive,Acer,maple,Fair,1733
1,Alive,Acer,maple,Good,4708
2,Alive,Acer,maple,Poor,639
3,Alive,Acer buergerianum,trident maple,Fair,20
4,Alive,Acer buergerianum,trident maple,Good,87


As the last modfication to our data,

In [ ]:
lab1_df_final = lab1_df_reduced_counts_grouped.merge(lab1_df_reduced_zerosFiltered_grouped, how="left")

In [ ]:
lab1_df_final = lab1_df_final.set_index(['status','spc_latin','spc_common','health'])
lab1_df_final

count  \
status spc_latin                          spc_common    health          
Alive  Acer                               maple         Fair     1733   
                                                        Good     4708   
                                                        Poor      639   
       Acer buergerianum                  trident maple Fair       20   
                                                        Good       87   
...                                                               ...   
       NaN                                NaN           Good        4   
                                                        Poor        1   
Dead   Gleditsia triacanthos var. inermis honeylocust   NaN         1   
       NaN                                NaN           NaN     13960   
Stump  NaN                                NaN           NaN     17654   

                                                                 diameter  
status spc_latin                          spc_common    health             
Alive  Acer                               maple         Fair    11.903991  
                                                        Good    11.778865  
                                                        Poor    10.509434  
       Acer buergerianum                  trident maple Fair     9.750000  
                                                        Good     7.724138  
...                                                                   ...  
       NaN                                NaN           Good    13.000000  
                                                        Poor    11.000000  
Dead   Gleditsia triacanthos var. inermis honeylocust   NaN     12.000000  
       NaN                                NaN           NaN      5.492017  
Stump  NaN                                NaN           NaN     16.750481  

[398 rows x 2 columns]

###**Finally, save the final DataFrame as a .csv file and submit it alongside your Google Colab notebook for grading.**###

In [ ]:
lab1_df_final.to_csv("Daniela_Cebotari_lab1_df_final.csv")